# Interpolação

#### Imports

In [ ]:
from numpy import zeros, concatenate, linspace
from matplotlib.pyplot import figure, rc
from numpy.typing import NDArray
from itertools import pairwise
from typing import Callable

#### Classe Interpolação

In [ ]:
function = Callable[[float | NDArray], float | NDArray]

class Interpolção:
  __slots__ = "D", "A", "f", "n", "I", "B", "zipper", "respol", "resnew"

  def __init__(self, func: function, inter: tuple[int, int], n: int, spl: int):
    (i1, i2), (div, mod) = inter, divmod(n, spl) if spl else (1, 0)
    self.D, self.A = linspace(i1, i2, 5000), linspace(i1, i2, n)
    self.f, self.n = div, (spl + 1) if (mod == 1) else n
    self.I, self.B = func(self.D), func(self.A)

    self.zipper = [*zip(self.spliter(self.D, 1),
                        self.spliter(self.A, 0),
                        self.spliter(self.B, 0))]

    self.interpol(); self.internew(); self.plotgfx()

  def spliter(self, array: NDArray, flag: bool):
    fator = (array.size / self.f) if flag else self.n
    a, b, c = fator, (array.size - fator), array.size
    points = linspace((0, a), (b, c), self.f, dtype = int)
    return (array[a:b] for a, b in points)

  def interpol(self): # Interpolação Polinomial
    def interpol(D: NDArray, A: NDArray, B: NDArray):
      L = zeros((self.n, self.n))
      L.flat[::self.n + 1] = 1
      U = A.reshape(-1, 1) ** range(self.n)

      for i1, i2 in pairwise(range(self.n)):
        L[i2:, i1] = U[i2:, i1] / U[i1, i1]
        U[i2:] -= L[i2:, i1, None] * U[i1]

      x, y = zeros(self.n), zeros(self.n)

      for i in range(self.n):
        y[i] = B[i] - (L[i] * y).sum()
      for i in range(self.n)[::-1]:
        x[i] = (y[i] - (U[i] * x).sum()) / U[i, i]

      return (D.reshape(-1, 1) ** range(self.n)) @ x

    lista = [interpol(*tupla) for tupla in self.zipper]
    self.respol = concatenate(lista)

  def internew(self): # Interpolação de Newton
    def internew(D: NDArray, A: NDArray, B: NDArray):
      M = zeros((self.n, self.n)); M[:, 0] = B

      for i1, i2 in pairwise(range(self.n)):
        temp = (M[i2:, i1] - M[i1:-1, i1])
        M[i2:, i2] = temp / (A[i2] - A[0])

      diag = M.diagonal()
      pontos = D.reshape(-1, 1) - A[:-1]
      return pontos.cumprod(1) @ diag[1:] + diag[0]

    lista = [internew(*tupla) for tupla in self.zipper]
    self.resnew = concatenate(lista)

  def plotgfx(self): # Plotagem das iterpolações
    rc("font", family = "Consolas", size = 7.5)
    rc("lines", aa = 1, lw = .5, markersize = 1)

    origin, pontos = "Original", "Interpolações"
    labels = ("Int. Polinomial", "Int. de Newton")
    imagens = (self.respol, self.resnew)

    fig = figure(figsize = (12, 3.5)); fig.dpi = 400
    axes = fig.subplots(1, len(imagens))

    for axe, label, imagem in zip(axes, labels, imagens):
      axe.plot(self.D, self.I, c = "k", label = origin)
      axe.scatter(self.A, self.B, c = "r", label = pontos)
      axe.plot(self.D, imagem, c = "m", label = label)
      axe.set_title("Comparação das Interpolações")
      axe.set_xlabel("Domínio das funções")
      axe.set_ylabel("Imagens das funções")
      axe.legend()

    fig.tight_layout(pad = 1)

#### Execução do Código

In [ ]:
Interpolção(lambda x: 1/(1 + 25*x**2), (-1, 1), 10, 0); pass

In [ ]:
Interpolção(lambda x: 1/(1 + 25*x**2), (-1, 1), 10, 3); pass